In [1]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
import pandas as pd
import sys
import time
from datetime import datetime

In [2]:
# Define function 
def check_exists_by_xpath(xpath, timeout=2):
    '''Return a boolean that indicates the presence of an element given a xpath.'''
    try:
        WebDriverWait(driver, timeout=timeout).until(
            EC.presence_of_element_located((By.XPATH, xpath)))
        return True
    except:
        return False

In [12]:
# Scrape specified pages
for n in range(26, 51, 5):
    start = n
    end = n+4
    print(start, end)

    start_time = time.time()
    print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

    options = webdriver.ChromeOptions()
    options.add_experimental_option('prefs', {
        "download.default_directory": 'C:\\Users\\user\\Dropbox\\0_MY498_Dissertation\\04_Codes\\data\\bernama\\articles', #Change default directory for downloads
        "download.prompt_for_download": False, # To auto download the file
        "download.directory_upgrade": True,
        "plugins.always_open_pdf_externally": True  #It will not show PDF directly in chrome
    })

    url = 'http://blis.bernama.com/index.php?mod=articles&opt=la&cid=1'
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    input_name = driver.find_element_by_id("txt_username")
    input_name.send_keys("Upustaka02")

    input_pw = driver.find_element_by_xpath("//input[@type='password']")
    input_pw.send_keys("Upustaka02")

    button_submit = driver.find_element_by_xpath("//button[@type='submit']")
    ActionChains(driver).click(button_submit).perform()
    titles = []
    n = 1

    while n <= end: 
        # Only scrape specified pages
        if n >= start:
            info = WebDriverWait(driver, timeout=10).until(
                EC.presence_of_all_elements_located((By.XPATH, "//td")))
            info = [title.text for title in info if title.text]
            info = [re.split('\n|\s\|\s', title.rstrip('|'))[:4] for title in info]
            titles.extend(info)

            rows = WebDriverWait(driver, timeout=10, ignored_exceptions=StaleElementReferenceException).until(
                EC.presence_of_all_elements_located((By.XPATH, "//tbody/tr[@role='row']")))

            for row in rows:
                # Click the view button to show articles' content
                button_view = row.find_element_by_xpath(".//td[2]/b/a[@class='view_article']")
                action = ActionChains(driver).click(button_view)
                action.pause(1)
                action.perform()

                # Click the download button to download pdf
                try:
                    button_download = WebDriverWait(driver, timeout=10).until(
                        EC.element_to_be_clickable((By.XPATH, "//a[@onclick='javascript: fn_go_download()']")))
                    action = ActionChains(driver).click(button_download)
                    action.pause(1)
                    action.perform()
                except:
                    pass

                # Click the close button to close the window
                button_close = WebDriverWait(driver, timeout=10).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[@class='btn btn-default close-modal']")))
                action = ActionChains(driver).move_to_element(button_close).click()
                action.pause(1)
                action.perform()

        rows = WebDriverWait(driver, timeout=10, ignored_exceptions=StaleElementReferenceException).until(
            EC.presence_of_element_located((By.XPATH, "//tbody/tr[@role='row']")))

        # Navigate to the next page
        button_next = WebDriverWait(driver, timeout=10).until(
            EC.element_to_be_clickable((By.XPATH, "//li[@class='paginate_button next']/a")))
        action = ActionChains(driver).click(button_next)
        action.pause(1)
        action.perform()

        # Wait until previous rows get stale
        wait = WebDriverWait(driver, timeout=10)
        wait.until(EC.staleness_of(rows)) 

    df = pd.DataFrame(titles)
    df.columns = ['title', 'category', 'date', 'author']

    time.sleep(3)
    button_logout = driver.find_element_by_xpath("//a[@title='Logout']")
    ActionChains(driver).click(button_logout).perform()

    df.to_csv('data/bernama/berita_dalam_negeri_' + str(start) + '_' + str(end) + '.csv', index=False, encoding='utf-8-sig')
    print("done", time.time()-start_time)
    driver.quit()

26 30
2021-07-18 01:45:03
done 614.7829959392548
31 35
2021-07-18 01:55:19
done 617.4964632987976
36 40
2021-07-18 02:05:39


ValueError: Length mismatch: Expected axis has 5 elements, new values have 4 elements

In [3]:
def scrape_articles():
    # Collect articles
    rows = WebDriverWait(driver, timeout=10, ignored_exceptions=StaleElementReferenceException).until(
        EC.presence_of_all_elements_located((By.XPATH, "//tbody/tr[@role='row']")))

    for row in rows:
        # Click the view button to show articles' content
        button_view = row.find_element_by_xpath(".//td[2]/b/a[@class='view_article']")
        action = ActionChains(driver).click(button_view)
        action.pause(1)
        action.perform()

        # Click the download button to download pdf
        try:
            button_download = WebDriverWait(driver, timeout=10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[@onclick='javascript: fn_go_download()']")))
            action = ActionChains(driver).click(button_download)
            action.pause(1)
            action.perform()
        except:
            pass

        # Click the close button to close the window
        try:
            button_close = WebDriverWait(driver, timeout=10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[@class='btn btn-default close-modal']")))
            action = ActionChains(driver).move_to_element(button_close).click()
            action.pause(1)
            action.perform()
        except:
            pass

In [6]:
# Scrape specified dates
start = '12/21/2017'
end = '12/31/2017'
year = '2017'
print(start, end)

dates = pd.date_range(start, end).strftime('%d-%m-%Y')
print('No of days:', len(dates))

for date in dates:
    print(date)
    start_time = time.time()
    print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

    options = webdriver.ChromeOptions()
    options.add_experimental_option('prefs', {
        "download.default_directory": 'C:\\Users\\user\\Dropbox\\0_MY498_Dissertation\\04_Codes\\data\\bernama\\articles\\' + year, #Change default directory for downloads
        "download.prompt_for_download": False, # To auto download the file
        "download.directory_upgrade": True,
        "plugins.always_open_pdf_externally": True  #It will not show PDF directly in chrome
    })

    url = 'http://blis.bernama.com/index.php?mod=articles&opt=la&cid=1'
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    input_name = driver.find_element_by_id("txt_username")
    input_name.send_keys("Upustaka01")

    input_pw = driver.find_element_by_xpath("//input[@type='password']")
    input_pw.send_keys("Upustaka01")

    button_submit = driver.find_element_by_xpath("//button[@type='submit']")
    ActionChains(driver).click(button_submit).perform()
    titles = []

    input_date = WebDriverWait(driver, timeout=10).until(EC.presence_of_element_located((By.XPATH, "//input[@id='articles_date']")))
    input_date.send_keys(date)
    input_date.send_keys(Keys.RETURN)

    next_page = True
    while next_page:
        # Collect metadata
        info = WebDriverWait(driver, timeout=10).until(EC.presence_of_all_elements_located((By.XPATH, "//td")))
        info = [title.text for title in info if title.text]
        info = [re.split('\n|\s\|\s', title.rstrip('|'))[:4] for title in info]
        titles.extend(info)

        # Collect articles
        scrape_articles()

        if check_exists_by_xpath("//li[@class='paginate_button next']/a", timeout=2):  
            rows = WebDriverWait(driver, timeout=10, ignored_exceptions=StaleElementReferenceException).until(
                EC.presence_of_element_located((By.XPATH, "//tbody/tr[@role='row']")))

            # Navigate to the next page
            button_next = WebDriverWait(driver, timeout=10).until(
                EC.element_to_be_clickable((By.XPATH, "//li[@class='paginate_button next']/a")))
            action = ActionChains(driver).click(button_next)
            action.pause(1)
            action.perform()

            # Wait until previous rows get stale
            wait = WebDriverWait(driver, timeout=10)
            wait.until(EC.staleness_of(rows)) 
        else:
            print('last page')
            next_page = False

    df = pd.DataFrame(titles)
    df.columns = ['title', 'category', 'date', 'author']

    time.sleep(3)
    button_logout = driver.find_element_by_xpath("//a[@title='Logout']")
    ActionChains(driver).click(button_logout).perform()

    df.to_csv('data/bernama/berita_dalam_negeri_' + date + '.csv', index=False, encoding='utf-8-sig')
    print("done", time.time()-start_time)
    driver.quit()

12/21/2017 12/31/2017
No of days: 11
21-12-2017
2021-07-21 17:20:13
last page
done 542.2931632995605
22-12-2017
2021-07-21 17:29:17
last page
done 469.7374348640442
23-12-2017
2021-07-21 17:37:09
last page
done 351.8384356498718
24-12-2017
2021-07-21 17:43:03
last page
done 224.62509751319885
25-12-2017
2021-07-21 17:46:50
last page
done 165.6943109035492
26-12-2017
2021-07-21 17:49:38
last page
done 383.84178495407104
27-12-2017
2021-07-21 17:56:04
last page
done 371.9591238498688
28-12-2017
2021-07-21 18:02:18
last page
done 451.98621678352356
29-12-2017
2021-07-21 18:09:52
last page
done 484.6522789001465
30-12-2017
2021-07-21 18:17:59
last page
done 322.0493688583374
31-12-2017
2021-07-21 18:23:23
last page
done 265.60854959487915


In [56]:
start_time = time.time()
print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    "download.default_directory": 'C:\\Users\\user\\Dropbox\\0_MY498_Dissertation\\04_Codes\\data\\bernama\\articles', #Change default directory for downloads
    "download.prompt_for_download": False, # To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True  #It will not show PDF directly in chrome
})

url = 'http://blis.bernama.com/index.php?mod=articles&opt=la&cid=1'
driver = webdriver.Chrome(options=options)
driver.get(url)

input_name = driver.find_element_by_id("txt_username")
input_name.send_keys("Upustaka01")

input_pw = driver.find_element_by_xpath("//input[@type='password']")
input_pw.send_keys("Upustaka01")

button_submit = driver.find_element_by_xpath("//button[@type='submit']")
ActionChains(driver).click(button_submit).perform()
titles = []
n = 1

dates = pd.date_range(start, end).strftime('%d/%m/%Y')
print('No of days:', len(dates))

for date in dates:
    input_date = WebDriverWait(driver, timeout=10).until(EC.presence_of_element_located((By.XPATH, "//input[@id='articles_date']")))
    input_date.clear()
    
    input_date = WebDriverWait(driver, timeout=10).until(EC.presence_of_element_located((By.XPATH, "//input[@id='articles_date']")))
    input_date.send_keys(date)
    input_date.send_keys(Keys.RETURN)

2021-07-18 04:22:15
No of days: 27


In [45]:
# Scrape specified dates

start = '01-01-2017'
end = '01-01-2017'
print(start, end)

start_time = time.time()
print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    "download.default_directory": 'C:\\Users\\user\\Dropbox\\0_MY498_Dissertation\\04_Codes\\data\\bernama\\articles', #Change default directory for downloads
    "download.prompt_for_download": False, # To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True  #It will not show PDF directly in chrome
})

url = 'http://blis.bernama.com/index.php?mod=articles&opt=la&cid=1'
driver = webdriver.Chrome(options=options)
driver.get(url)

input_name = driver.find_element_by_id("txt_username")
input_name.send_keys("Upustaka01")

input_pw = driver.find_element_by_xpath("//input[@type='password']")
input_pw.send_keys("Upustaka01")

button_submit = driver.find_element_by_xpath("//button[@type='submit']")
ActionChains(driver).click(button_submit).perform()
titles = []
n = 1

dates = pd.date_range(start, end).strftime('%d/%m/%Y')
print('No of days:', len(dates))

for date in dates:
    input_date = WebDriverWait(driver, timeout=10).until(EC.presence_of_element_located((By.XPATH, "//input[@id='articles_date']")))
    input_date.send_keys(date)
    input_date.send_keys(Keys.RETURN)

    next_page = True
    while next_page:
        # Collect metadata
        info = WebDriverWait(driver, timeout=10).until(EC.presence_of_all_elements_located((By.XPATH, "//td")))
        info = [title.text for title in info if title.text]
        info = [re.split('\n|\s\|\s', title.rstrip('|'))[:4] for title in info]
        titles.extend(info)
            
        # Collect articles
        #scrape_articles()
        
        if check_exists_by_xpath("//li[@class='paginate_button next']/a", timeout=2):  
            rows = WebDriverWait(driver, timeout=10, ignored_exceptions=StaleElementReferenceException).until(
                EC.presence_of_element_located((By.XPATH, "//tbody/tr[@role='row']")))

            # Navigate to the next page
            n += 1
            button_next = WebDriverWait(driver, timeout=10).until(
                EC.element_to_be_clickable((By.XPATH, "//li[@class='paginate_button next']/a")))
            action = ActionChains(driver).click(button_next)
            action.pause(1)
            action.perform()

            # Wait until previous rows get stale
            wait = WebDriverWait(driver, timeout=10)
            wait.until(EC.staleness_of(rows)) 
        else:
            print('last page')
            
            # Collect metadata
            info = WebDriverWait(driver, timeout=10).until(EC.presence_of_all_elements_located((By.XPATH, "//td")))
            info = [title.text for title in info if title.text]
            info = [re.split('\n|\s\|\s', title.rstrip('|'))[:4] for title in info]
            titles.extend(info)

            # Collect articles
            #scrape_articles()
            next_page = False
            
df = pd.DataFrame(titles)
df.columns = ['title', 'category', 'date', 'author']

time.sleep(3)
button_logout = driver.find_element_by_xpath("//a[@title='Logout']")
ActionChains(driver).click(button_logout).perform()

df.to_csv('data/bernama/berita_dalam_negeri_' + str(start) + '_' + str(end) + '.csv', index=False, encoding='utf-8-sig')
print("done", time.time()-start_time)
driver.quit()

01-01-2017 01-01-2017
2021-07-18 03:24:51
No of days: 1
last page
done 25.38996434211731


In [44]:
start = '01-01-2017'
end = '01-01-2017'
pd.date_range(start, end).strftime('%d/%m/%Y')

Index(['01/01/2017'], dtype='object')

In [13]:
start = 36
end = 40
print(start, end)

start_time = time.time()
print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    "download.default_directory": 'C:\\Users\\user\\Dropbox\\0_MY498_Dissertation\\04_Codes\\data\\bernama\\articles', #Change default directory for downloads
    "download.prompt_for_download": False, # To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True  #It will not show PDF directly in chrome
})

url = 'http://blis.bernama.com/index.php?mod=articles&opt=la&cid=1'
driver = webdriver.Chrome(options=options)
driver.get(url)

input_name = driver.find_element_by_id("txt_username")
input_name.send_keys("Upustaka02")

input_pw = driver.find_element_by_xpath("//input[@type='password']")
input_pw.send_keys("Upustaka02")

button_submit = driver.find_element_by_xpath("//button[@type='submit']")
ActionChains(driver).click(button_submit).perform()
titles = []
n = 1

while n <= end: 
    # Only scrape specified pages
    if n >= start:
        info = WebDriverWait(driver, timeout=10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//td")))
        info = [title.text for title in info if title.text]
        info = [re.split('\n|\s\|\s', title.rstrip('|'))[:4] for title in info]
        titles.extend(info)
        
    rows = WebDriverWait(driver, timeout=10, ignored_exceptions=StaleElementReferenceException).until(
        EC.presence_of_element_located((By.XPATH, "//tbody/tr[@role='row']")))

    # Navigate to the next page
    n += 1
    button_next = WebDriverWait(driver, timeout=10).until(
        EC.element_to_be_clickable((By.XPATH, "//li[@class='paginate_button next']/a")))
    action = ActionChains(driver).click(button_next)
    action.pause(1)
    action.perform()

    # Wait until previous rows get stale
    wait = WebDriverWait(driver, timeout=10)
    wait.until(EC.staleness_of(rows)) 

df = pd.DataFrame(titles)
df.columns = ['title', 'category', 'date', 'author']

time.sleep(3)
button_logout = driver.find_element_by_xpath("//a[@title='Logout']")
ActionChains(driver).click(button_logout).perform()

df.to_csv('data/bernama/berita_dalam_negeri_' + str(start) + '_' + str(end) + '.csv', index=False, encoding='utf-8-sig')
print("done", time.time()-start_time)
driver.quit()

36 40
2021-07-18 02:17:31
done 121.83046317100525


In [14]:
options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    "download.default_directory": 'C:\\Users\\user\\Dropbox\\0_MY498_Dissertation\\04_Codes\\data\\bernama\\articles', #Change default directory for downloads
    "download.prompt_for_download": False, # To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True  #It will not show PDF directly in chrome
})

url = 'http://blis.bernama.com/index.php?mod=articles&opt=la&cid=1'
driver = webdriver.Chrome(options=options)
driver.get(url)

input_name = driver.find_element_by_id("txt_username")
input_name.send_keys("Upustaka02")

input_pw = driver.find_element_by_xpath("//input[@type='password']")
input_pw.send_keys("Upustaka02")

button_submit = driver.find_element_by_xpath("//button[@type='submit']")
ActionChains(driver).click(button_submit).perform()